# ELAS – Colab (avancé): Pantheon+ + DESI DR2 + low‑z BAO + Planck 2018

Ce notebook génère les jeux de données BAO/CMB, implémente ΛCDM/ELAS, et calcule χ² totaux.

In [1]:

import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy import integrate
c = 299792.458
print("Imports OK")


Imports OK


## 1) Jeux de données (création locale)

In [2]:

import pandas as pd
planck_means = pd.DataFrame({"parameter":["R","l_a","omega_b_h2"],"mean":[1.750235,301.4707,0.02235976]})
planck_inv_cov = pd.DataFrame([[94392.3971,-1360.4913,1664517.2916],[-1360.4913,161.4349,3671.6180],[1664517.2916,3671.6180,79719182.5162]],
                              columns=["R","l_a","omega_b_h2"], index=["R","l_a","omega_b_h2"])
planck_means.to_csv("planck2018_distance_priors_means.csv", index=False)
planck_inv_cov.to_csv("planck2018_distance_priors_inv_cov.csv")
bao_lowz = pd.DataFrame([{"survey":"6dFGS","z_eff":0.106,"quantity":"rs_over_DV","value":0.336,"sigma":0.015},
                         {"survey":"SDSS_MGS","z_eff":0.15,"quantity":"DV_over_rd_scaled","value":664.0,"sigma":25.0}])
bao_lowz.to_csv("bao_lowz_6dfgs_sdssmgs.csv", index=False)
desi = pd.DataFrame([["LRG1",0.510,13.588,0.167,21.863,0.425,-0.459],
                     ["LRG2",0.706,17.351,0.177,19.455,0.330,-0.404],
                     ["LRG3+ELG1",0.934,21.576,0.152,17.641,0.193,-0.416],
                     ["ELG2",1.321,27.601,0.318,14.176,0.221,-0.434],
                     ["QSO",1.484,30.512,0.760,12.817,0.516,-0.500],
                     ["LyA",2.330,38.988,0.531,8.632,0.101,-0.431]],
                    columns=["tracer","z_eff","DM_over_rd","DM_sigma","DH_over_rd","DH_sigma","corr_r"])
desi_bgs = pd.DataFrame([{"tracer":"BGS","z_eff":0.295,"DV_over_rd":7.942,"DV_sigma":0.075}])
desi.to_csv("desi_dr2_bao_tableIV_core.csv", index=False)
desi_bgs.to_csv("desi_dr2_bao_tableIV_bgs.csv", index=False)
print("Fichiers BAO/CMB écrits.")


Fichiers BAO/CMB écrits.


## 2) Cosmologie : ΛCDM & ELAS

In [3]:

import numpy as np
from scipy import integrate

def Ez_LCDM(z, H0=67.4, Omega_m=0.315):
    return np.sqrt(Omega_m*(1+z)**3 + (1-Omega_m))

def Ez_ELAS(z, H0=67.4, Omega_m=0.315, delta=0.05, Omega_osc=2.0, phi=-2.62):
    base = Omega_m*(1+z)**3 + (1-Omega_m)*(1 + delta*np.cos(Omega_osc*np.log(1+z) + phi))
    return np.sqrt(np.maximum(base,1e-12))

def make_Ez(model="LCDM", H0=67.4, Omega_m=0.315, delta=0.05, Omega_osc=2.0, phi=-2.62):
    return (lambda z: Ez_LCDM(z,H0,Omega_m)) if model=="LCDM" else (lambda z: Ez_ELAS(z,H0,Omega_m,delta,Omega_osc,phi))

def DM_shape(z, Ez):
    z = np.atleast_1d(z); out=[]
    for zi in z:
        val = integrate.quad(lambda zz: 1.0/max(Ez(zz),1e-12), 0, zi, limit=200)[0]
        out.append(val)
    return np.array(out)

def DH_shape(z, Ez): return 1.0/np.array(Ez(z))
def DV_shape(z, Ez):
    z = np.atleast_1d(z); DH = DH_shape(z, Ez); DM = DM_shape(z, Ez); return (z*DH*DM**2)**(1/3)
def mu_from_DL(z, Ez, H0=67.4):
    DM = DM_shape(z, Ez)*(c/H0); DL=(1+np.atleast_1d(z))*DM; return 5*np.log10(np.maximum(DL,1e-12))+25
print("Fonctions OK")


Fonctions OK


## 3) χ² (CMB / BAO / SN) & Δμ(z)

In [4]:

import pandas as pd, numpy as np, matplotlib.pyplot as plt

def cmb_distance_priors_theory(H0=67.4, Omega_m=0.315, Omega_b_h2=0.02237):
    zstar=1089.0; Ez=make_Ez("LCDM",H0,Omega_m); DM=DM_shape(zstar,Ez)*(c/H0); rs=147.09
    R=np.sqrt(Omega_m)*(H0/100.)*DM/(c/100.); l_a=np.pi*DM/rs; return float(R),float(l_a),float(Omega_b_h2)

def chi2_cmb_from_priors(H0=67.4, Omega_m=0.315, Omega_b_h2=0.02237):
    means=pd.read_csv("planck2018_distance_priors_means.csv"); inv=pd.read_csv("planck2018_distance_priors_inv_cov.csv",index_col=0)
    theo=np.array(cmb_distance_priors_theory(H0,Omega_m,Omega_b_h2)); data=means["mean"].values; diff=theo-data; icov=inv.values
    return float(diff@icov@diff)

desi=pd.read_csv("desi_dr2_bao_tableIV_core.csv"); desi_bgs=pd.read_csv("desi_dr2_bao_tableIV_bgs.csv"); lowz=pd.read_csv("bao_lowz_6dfgs_sdssmgs.csv")

def chi2_bao_desi(H0=67.4, Omega_m=0.315, model="LCDM", delta=0.05, Omega_osc=2.0, phi=-2.62):
    Ez=make_Ez(model,H0,Omega_m,delta,Omega_osc,phi); z=desi["z_eff"].values
    DM_th=DM_shape(z,Ez); DH_th=DH_shape(z,Ez)
    DM_obs=desi["DM_over_rd"].values; DH_obs=desi["DH_over_rd"].values
    sDM=desi["DM_sigma"].values; sDH=desi["DH_sigma"].values; r=desi["corr_r"].values
    chi2=0.0
    for i in range(len(z)):
        cov=np.array([[sDM[i]**2, r[i]*sDM[i]*sDH[i]],[r[i]*sDM[i]*sDH[i], sDH[i]**2]]); inv=np.linalg.inv(cov)
        d=np.array([DM_th[i]-DM_obs[i], DH_th[i]-DH_obs[i]]); chi2+=float(d@inv@d)
    zb=float(desi_bgs["z_eff"].values[0]); DV_th=float(DV_shape(zb,Ez)); DV_obs=float(desi_bgs["DV_over_rd"].values[0]); sDV=float(desi_bgs["DV_sigma"].values[0])
    chi2+=((DV_th-DV_obs)/sDV)**2
    row=lowz[lowz["survey"]=="6dFGS"].iloc[0]; z6=row["z_eff"]; val=row["value"]; sig=row["sigma"]; DV6=float(DV_shape(z6,Ez)); rs_over_DV_th=1.0/DV6
    chi2+=((rs_over_DV_th - val)/sig)**2
    return chi2

def chi2_sn_from_file(path, z_col="zCMB", mu_col="MU_SH0ES", sigma_col="MU_SH0ES_ERR_DIAG",
                      H0=67.4, Omega_m=0.315, model="LCDM", delta=0.05, Omega_osc=2.0, phi=-2.62):
    df=pd.read_csv(path); z=df[z_col].values; mu=df[mu_col].values; sig=df[sigma_col].values
    Ez=make_Ez(model,H0,Omega_m,delta,Omega_osc,phi); mu_th=mu_from_DL(z,Ez,H0)
    w=1/np.maximum(sig,1e-9)**2; a=np.sum((mu-mu_th)**2*w); b=np.sum((mu-mu_th)*w); c=np.sum(w); return float(a-b*b/c)

def chi2_total(pantheon_path=None, H0=67.4, Omega_m=0.315, model="LCDM", delta=0.05, Omega_osc=2.0, phi=-2.62,
               include_cmb=True, include_bao=True):
    chi2=0.0
    if include_cmb: chi2+=chi2_cmb_from_priors(H0,Omega_m)
    if include_bao: chi2+=chi2_bao_desi(H0,Omega_m,model,delta,Omega_osc,phi)
    if pantheon_path: chi2+=chi2_sn_from_file(pantheon_path,H0=H0,Omega_m=Omega_m,model=model,delta=delta,Omega_osc=Omega_osc,phi=phi)
    return chi2

def plot_delta_mu(zmin=0.001,zmax=2.3,H0=67.4,Omega_m=0.315,delta=0.05,Omega_osc=2.0,phi=-2.62,N=200):
    z=np.linspace(zmin,zmax,N); EzL=make_Ez("LCDM",H0,Omega_m); EzE=make_Ez("ELAS",H0,Omega_m,delta,Omega_osc,phi)
    muL=mu_from_DL(z,EzL,H0); muE=mu_from_DL(z,EzE,H0); d=muE-muL
    plt.figure(figsize=(7,4.2)); plt.plot(z,d,lw=2); plt.axhline(0,ls="--"); plt.xlabel("z"); plt.ylabel("Δμ (mag)"); plt.title("ELAS vs ΛCDM"); plt.tight_layout(); plt.show()

print("χ² & plotting OK")


χ² & plotting OK


## 4) Scan grille simple (optionnel)

In [5]:

def grid_scan(pantheon_path=None, grid_delta=np.linspace(0.0,0.10,11),
              grid_Omega=np.linspace(1.0,4.0,16), grid_phi=np.linspace(-np.pi,np.pi,25),
              H0=67.4, Omega_m=0.315, include_cmb=True, include_bao=True):
    best={"chi2":np.inf,"delta":None,"Omega":None,"phi":None}
    for d in grid_delta:
        for Om in grid_Omega:
            chis=[]
            for ph in np.linspace(-np.pi,np.pi,9):
                c2=chi2_total(pantheon_path,H0,Omega_m,"ELAS",d,Om,ph,include_cmb,include_bao)
                chis.append((c2,ph))
            c2_best,ph_best=min(chis,key=lambda x:x[0])
            if c2_best<best["chi2"]:
                best.update({"chi2":float(c2_best),"delta":float(d),"Omega":float(Om),"phi":float(ph_best)})
    return best

print("grid_scan prêt")


grid_scan prêt
